In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline

import openmc
import openmc.mgxs as mgxs

In [2]:
temperatures = [922, 972, 1022, 1072, 1122]

###################
#### MATERIALS ####
###################

# Create the fuel Material
fuel_material = openmc.Material(1, "molten_salt_fuel")
fuel_material.add_nuclide('Li7', 0.1090, 'wo')
fuel_material.add_nuclide('Li6', 5e-6, 'wo')
fuel_material.add_nuclide('F19', 0.6680, 'wo')
fuel_material.add_nuclide('Be9', 0.0627, 'wo')
fuel_material.add_nuclide('U235', 0.0167, 'wo')
fuel_material.add_nuclide('U238', 0.0344, 'wo')
fuel_material.set_density('g/cm3', 2.146) # see Moltres paper

# Create the moderator Material
moderator_material = openmc.Material(2, "moderator")
moderator_material.add_element('C', 1, 'wo')
moderator_material.set_density('g/cm3', 1.86) # see Moltres paper
moderator_material.add_s_alpha_beta('c_Graphite')

# Create the Materials and export to XML
msre_materials = openmc.Materials([fuel_material, moderator_material])
msre_materials.export_to_xml()

In [3]:
##################
#### GEOMETRY ####
##################

# Reactor_core parameters
R = 72.5 # [cm], radius of the reactor reactor_core
H = 151.75 # [cm], height of the reactor reactor_core
num_segments = 14 # fuel and moderator segments
pitch = R / num_segments
x = 0.2379522 # what is this???
fuel_radius = x * pitch
moderator_radius = pitch - fuel_radius

# Create the reactor core Region
reactor_core_top_surface = openmc.ZPlane(z0=H, boundary_type='vacuum')
reactor_core_bottom_surface = openmc.ZPlane(z0=0, boundary_type='vacuum')
reactor_core_left_surface = openmc.XPlane(x0=0, boundary_type='reflective')
reactor_core_right_surface = openmc.XPlane(x0=R, boundary_type='vacuum')
reactor_core_region = -reactor_core_top_surface & +reactor_core_bottom_surface & +reactor_core_left_surface & -reactor_core_right_surface


# Create the reactor core Universe
reactor_core_universe = openmc.Universe(name='reactor_core')

# Create a fuel channel Region
fuel_channel_left_surface_coord = 0
fuel_channel_right_surface_coord = fuel_radius #might need to change this to be the full diameter
fuel_channel_left_surface = openmc.XPlane(x0 = fuel_channel_left_surface_coord)
fuel_channel_right_surface = openmc.XPlane(x0 = fuel_channel_right_surface_coord)
fuel_channel_region = +fuel_channel_left_surface & -fuel_channel_right_surface & reactor_core_region

# Create a fuel channel Cell
fuel_channel_cell = openmc.Cell(name='fuel_channel_0') 
fuel_channel_cell.fill = fuel_material
fuel_channel_cell.region = fuel_channel_region

# Create a moderator Region
moderator_channel_left_surface_coord = fuel_radius
moderator_channel_right_surface_coord = fuel_radius + moderator_radius#2 * moderator_radius
moderator_channel_left_surface = openmc.XPlane(moderator_channel_left_surface_coord)
moderator_channel_right_surface = openmc.XPlane(moderator_channel_right_surface_coord)
moderator_channel_region = +moderator_channel_left_surface & -moderator_channel_right_surface & reactor_core_region

# Create a moderator Cell
moderator_channel_cell = openmc.Cell(name='moderator_channel_0') 
moderator_channel_cell.fill = moderator_material
moderator_channel_cell.region = moderator_channel_region

# Add the fuel and moderator channel Cells to the reactor core Universe
reactor_core_universe.add_cell(fuel_channel_cell)
reactor_core_universe.add_cell(moderator_channel_cell)
cells = []
cells.append(fuel_channel_cell)
cells.append(moderator_channel_cell)

# Repeat above for the remaining 13 channels
for i in range(1,14):
    fuel_channel_region = fuel_channel_region.translate([pitch, 0, 0])
    
    fuel_channel_cell = openmc.Cell(name=f'fuel_channel_{i}') 
    fuel_channel_cell.fill = fuel_material
    fuel_channel_cell.region = fuel_channel_region
    
    moderator_channel_region = moderator_channel_region.translate([pitch, 0, 0])
    
    moderator_channel_cell = openmc.Cell(name=f'moderator_channel_{i}') 
    moderator_channel_cell.fill = moderator_material
    moderator_channel_cell.region = moderator_channel_region
    
    reactor_core_universe.add_cell(fuel_channel_cell)
    reactor_core_universe.add_cell(moderator_channel_cell)
    cells.append(fuel_channel_cell)
    cells.append(moderator_channel_cell)

#Create the root Cell
root_cell = openmc.Cell(name = 'root_cell', fill=reactor_core_universe)
root_cell.region = reactor_core_region

# Create the root Universe
root_universe = openmc.Universe(universe_id = 0, name='root_universe')
root_universe.add_cell(root_cell)

# Create the Geometry and export to XML
geometry = openmc.Geometry(root_universe)
geometry.export_to_xml()

In [4]:
##################
#### SETTINGS ####
##################

# OpenMC simulation parameters
batches = 50
inactive = 10
particles = 2500

# Instatiate a Settings Object
settings_file = openmc.Settings()
settings_file.batches = batches
settings_file.inactive = inactive
settings_file.particles = particles
settings_file.output = {'tallies' : True}

# Create an initial uniform spatial source distribution over fissionable zones
dist_bounds = [0, 0, 0, R, H, R]
uniform_dist = openmc.stats.Box(dist_bounds[:3], dist_bounds[3:], only_fissionable=True)
settings_file.source = openmc.Source(space=uniform_dist)

# Export Settings to XML
settings_file.export_to_xml()

In [5]:
#######################
#### ENERGY GROUPS ####
#######################

# Instantiate a 2-group EnergyGroups object
neutron_groups = mgxs.EnergyGroups([0., 0.625, 20.0e6]) #are these right? 

# Initialize a 2-energy-group and 6-delayed-group MGXS Library
mgxs_lib = mgxs.Library(geometry)
mgxs_lib.energy_groups = neutron_groups
mgxs_lib.num_delayed_groups = 6


# Specify multi-group cross section types to compute
mgxs_lib.mgxs_types = ['total', 'chi', 'chi-delayed', 'decay-rate', 'transport', 'nu-scatter matrix', 
                       'kappa-fission', 'scatter matrix', 'nu-fission', 'inverse-velocity',
                       'absorption', 'scatter', 'nu-scatter', 'fission']

# Specify a "material" domain type for the cross section tally filters
mgxs_lib.domain_type = 'material'

# Specify the cell domain over which to compute multi-group cross sections
mgxs_lib.domain = geometry.get_all_materials().values()

# Set the Legendre order to 3 for P3 scattering
#mgxs_lib.legendre_order = 3

In [6]:
# Check the library - if no errors are raised, then the library is satisfactory.
mgxs_lib.check_library_for_openmc_mgxs()

In [7]:
# Construct all tallies needed forthe multi-group cross section library
mgxs_lib.build_library()

# Create a "tallies.xml" file for the MGXS Library
tallies_file = openmc.Tallies()
mgxs_lib.add_to_tallies_file(tallies_file, merge=True)

# Instantiate a current tally
cell_filter = openmc.CellFilter(cells)
current_tally = openmc.Tally(name='current_tally')
current_tally.scores = ['current']
current_tally.filters = [cell_filter]

# Add current tally to the tallies file
tallies_file.append(current_tally)

# Export to "tallies.xml"
tallies_file.export_to_xml()

/home/ooblack/anaconda3/envs/nuclear/lib/python3.7/site-packages/openmc/mixin.py:71: IDWarning: Another Filter instance already exists with id=106.
  warn(msg, IDWarning)
/home/ooblack/anaconda3/envs/nuclear/lib/python3.7/site-packages/openmc/mixin.py:71: IDWarning: Another Filter instance already exists with id=5.
  warn(msg, IDWarning)
/home/ooblack/anaconda3/envs/nuclear/lib/python3.7/site-packages/openmc/mixin.py:71: IDWarning: Another Filter instance already exists with id=4.
  warn(msg, IDWarning)
/home/ooblack/anaconda3/envs/nuclear/lib/python3.7/site-packages/openmc/mixin.py:71: IDWarning: Another Filter instance already exists with id=9.
  warn(msg, IDWarning)
/home/ooblack/anaconda3/envs/nuclear/lib/python3.7/site-packages/openmc/mixin.py:71: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/anaconda3/envs/nuclear/lib/python3.7/site-packages/openmc/mixin.py:71: IDWarning: Another Filter instance already exists with id=16.
  warn

In [ ]:
###############
#### PLOTS ####
###############

# Instantiate a Plot object
plot = openmc.Plot()

# Plot parameters
plot.basis = 'xz'
plot.origin = (R/2, 0, H/2)
plot.width = (1.5*R, 1.5*H)
plot.pixels = (int(10*R), int(10*H))
plot.color_by = 'material'

# Create a Plots object and export to XML
plots = openmc.Plots([plot])
plots.export_to_xml()

# Plot the geometry
openmc.plot_geometry()

In [8]:
# Run OpenMC
openmc.run(threads=8)

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [9]:
# Move the statepoint File
ce_spfile = './statepoint_ce.h5'
os.rename('statepoint.' + str(batches) + '.h5', ce_spfile)
# Move the Summary file
ce_sumfile = './summary_ce.h5'
os.rename('summary.h5', ce_sumfile)

In [10]:
# Load the statepoint file
sp = openmc.StatePoint(ce_spfile, autolink=False)

# Load the summary file in its new location
su = openmc.Summary(ce_sumfile)
sp.link_with_summary(su)

In [11]:
# Initialize MGXS Library with OpenMC statepoint data
mgxs_lib.load_from_statepoint(sp)

In [12]:
# Create a MGXS File which can then be written to disk
mgxs_file = mgxs_lib.create_mg_library(xs_type='macro', xsdata_names=['molten_salt_fuel', 'moderator'])

# Write the file to disk using the default filename of "mgxs.h5"
mgxs_file.export_to_hdf5()